# <center> Introduction To Gevent</center>


<center><img align="center" src="imgs/gevent_cover.png"></center>

# <center>简介</center>

- 基于 libev 的并发库
- Greenlet, 它是以C扩展模块形式接入Python的轻量级协程
- Greenlet全部运行在主程序操作系统进程的内部，但它们被协作式地调度。在任何时刻，只有一个协程在运行


# <center>简介</center>

- 并发的核心思想在于，大的任务可以分解成一系列的子任务，后者可以被调度成同时执行或异步执行
- 两个子任务之间的 切换也就是上下文切换
- 在gevent里面，上下文切换是通过yielding来完成的

In [8]:
import gevent

def foo():
    print('Running in foo')
    gevent.sleep(0)
    print('Explicit context switch to foo again')

def bar():
    print('Explicit context to bar')
    gevent.sleep(0)
    print('Implicit context switch back to bar')

gevent.joinall([
    gevent.spawn(foo),
    gevent.spawn(bar),
])
print('hehe')

Running in foo
Explicit context to bar
Explicit context switch to foo again
Implicit context switch back to bar
hehe


<center><img align="center" src="imgs/gevent_yield.gif"></center>

# <center>协作式调度</center>

- 网络IO中 gevent 负责协作式调度 greenlet
- 不用关心底层细节就能实现网络并发

In [12]:
import gevent
import random

def task(pid):
    gevent.sleep(random.randint(0,2)*0.001)
    print('Task %s done' % pid)

def synchronous():
    for i in range(1,10):
        task(i)

def asynchronous():
    threads = [gevent.spawn(task, i) for i in range(10)]
    gevent.joinall(threads)

print('Synchronous:')
synchronous()

print('Asynchronous:')
asynchronous()

Synchronous:
Task 1 done
Task 2 done
Task 3 done
Task 4 done
Task 5 done
Task 6 done
Task 7 done
Task 8 done
Task 9 done
Asynchronous:
Task 3 done
Task 5 done
Task 1 done
Task 4 done
Task 6 done
Task 8 done
Task 9 done
Task 0 done
Task 2 done
Task 7 done


In [21]:
import gevent.monkey
gevent.monkey.patch_socket()

import gevent
import requests
import json, time

def fetch(pid):
    response = requests.get('http://www.baidu.com')

def synchronous():
    for i in range(1,10):
        fetch(i)

def asynchronous():
    threads = []
    for i in range(1,10):
        threads.append(gevent.spawn(fetch, i))
    gevent.joinall(threads)
beg = time.time()
print('Synchronous:')
synchronous()
print(time.time()-beg)

beg = time.time()
print('Asynchronous:')
asynchronous()
print(time.time()-beg)

Synchronous:
0.21242594718933105
Asynchronous:
0.06549978256225586


# <center>创建 Greenlets</center>

- 使用 Greenlet.spawn
- 子类化 Greenlet类，重写 _run 方法

In [23]:
import gevent
from gevent import Greenlet

def foo(message, n):
    gevent.sleep(n)
    print(message)

# Initialize a new Greenlet instance running the named function
# foo
thread1 = Greenlet.spawn(foo, "Hello", 1)

# Wrapper for creating and running a new Greenlet from the named
# function foo, with the passed arguments
thread2 = gevent.spawn(foo, "I live!", 2)

# Lambda expressions
thread3 = gevent.spawn(lambda x: (x+1), 2)

threads = [thread1, thread2, thread3]

# Block until all threads complete.
gevent.joinall(threads)

Hello
I live!


[<Greenlet "Greenlet-6" at 0x10584f048: _run>,
 <Greenlet "Greenlet-7" at 0x104c5c148: _run>,
 <Greenlet "Greenlet-8" at 0x104c5cd48: _run>]

In [22]:
import gevent
from gevent import Greenlet

class MyGreenlet(Greenlet):

    def __init__(self, message, n):
        Greenlet.__init__(self)
        self.message = message
        self.n = n

    def _run(self):
        print(self.message)
        gevent.sleep(self.n)

g = MyGreenlet("Hi there!", 3)
g.start()
g.join()

Hi there!


# <center>Greenlet 状态</center>

- started -- Boolean, 指示此Greenlet是否已经启动
- ready() -- Boolean, 指示此Greenlet是否已经停止
- successful() -- Boolean, 指示此Greenlet是否已经停止而且没抛异常
- value -- 任意值, 此Greenlet代码返回的值
- exception -- 异常, 此Greenlet内抛出的未捕获异常

In [ ]:
import gevent

def win():
    return 'You win!'

def fail():
    raise Exception('You fail at failing.')

winner = gevent.spawn(win)
loser = gevent.spawn(fail)

print(winner.started) # True
print(loser.started)  # True

# Exceptions raised in the Greenlet, stay inside the Greenlet.
try:
    gevent.joinall([winner, loser])
except Exception as e:
    print('This will never be reached')

print(winner.value) # 'You win!'
print(loser.value)  # None

print(winner.ready()) # True
print(loser.ready())  # True

print(winner.successful()) # True
print(loser.successful())  # False

# The exception raised in fail, will not propogate outside the
# greenlet. A stack trace will be printed to stdout but it
# will not unwind the stack of the parent.

print(loser.exception)

# <center>猴子补丁(Monkey Patching)</center>

- 黑魔法，运行时属性替换。有用的邪恶(useful evil)
- gevent能够 修改标准库里面大部分的阻塞式系统调用，包括socket、ssl、threading和 select等模块，而变为协作式运行
- from gevent import monkey; monkey.patch_socket()

In [ ]:
import socket
print(socket.socket)

print("After monkey patch")
from gevent import monkey
monkey.patch_socket()
print(socket.socket)

import select
print(select.select)
monkey.patch_select()
print("After monkey patch")
print(select.select)

# <center>数据结构</center>

- event, 在Greenlet之间异步通信的形式
- Queue, 队列是一个排序的数据集合，它有常见的put / get操作， 但是它是以在Greenlet之间可以安全操作的方式来实现的
- group, 是一个运行中greenlet的集合，集合中的greenlet像一个组一样 会被共同管理和调度
- pool, 是一个为处理数量变化并且需要限制并发的greenlet而设计的结构。 在需要并行地做很多受限于网络和IO的任务时常常需要用到它
- BoundedSemaphore, 信号量, 允许greenlet相互合作，限制并发访问或运行的低层次的同步原语， 信号量有两个方法，acquire和release
- greenlet，允许你指定局部于greenlet上下文的数据

# <center>其他模块</center>

- StreamServer
- WSGI Servers
- websocket

# <center>参考</center>

- [Gevent 程序员指南](http://ningning.today/gevent-tutorial-cn/)